# k-Armed Bandit

- The *value* is the *expected reward*
  $$
  \begin{align*}
  q_\star(a) \doteq& \textbf{E}[R_t|A_t=a]\quad\forall a\in\{1,\cdots,k\}\\
  =& \sum_r{p(r|a)r}
  \end{align*}
  $$
- the goal is to *maximize* the *expected reward*
  $$
  \underset{a}{\mathrm{argmax}}\ q_\star(a)
  $$